# MS Semantic Kernel Memory - Knowledge Base for Information Worker

## Intro



### Step 1: Azure Environment

This [Azure CLI script](../CreateEnv/CreateEnv.azcli) creates: 
- an Azure Open AI instance
- deploys `text-embedding-ada-002` to calculate embeddings
- deploys `gpt-35-turbo` to summarize wikipedia articles (knowledge)

The script provides necessary credentials (API key), endpoint information (Azure OpenAI) and model deployment names (embedding & chat completion) and stores them in environment variables.

```azurecli
$ENV:AZURE_OPENAI_ENDPOINT = $csEndpoint
$ENV:AZURE_OPENAI_API_KEY = $csApiKey
$ENV:AZURE_OPENAI_CHATCOMPLETION_DEPLOYMENTNAME = $modelChatCompletionDeploymentName
$ENV:AZURE_OPENAI_EMBEDDING_DEPLOYMENTNAME = $modelEmbeddingDeploymentName
```

### Step 2: Build Semantic Kernel Instance

A Semantic Kernel instance is created with:
- Azure chat completion service
- Azure embedding service
- Memory storage (VolatileMemoryStorage in this example)

In [2]:
#r "nuget: Microsoft.SemanticKernel, 0.16.230615.1-preview"

using Microsoft.SemanticKernel; 
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.SemanticFunctions;
using System.Text.Json;

//Define Azure OpenAI information
string _apiEndpoint = "<<Your Azure OpenAI API endpoint>>";
string _apiKey = "<<Your Azure OpenAI API key>>";
string _modelChatCompletionDeploymentName = "<<your Azure OpenAI chat completion deployment name>>";
string _modelEmbeddingDeploymentName = "<<your Azure OpenAI embedding deployment name>>";


//Build Semantic Kernel
var kernelBuilder = new KernelBuilder()
    .WithAzureChatCompletionService(
        apiKey: _apiKey, 
        endpoint: _apiEndpoint, 
        deploymentName: _modelChatCompletionDeploymentName
    )
    .WithAzureTextEmbeddingGenerationService(
        endpoint: _apiEndpoint,
        apiKey: _apiKey,
        deploymentName: _modelEmbeddingDeploymentName
    )
    .WithMemoryStorage(new VolatileMemoryStore());
IKernel kernel = kernelBuilder.Build();

#!share --from c# kernel --as kernel


Installed Packages Microsoft.SemanticKernel, 0.16.230615.1-preview

### Step 3: Define inline Skill 

To summarize knowledge from sources a semantic function `SimpleSummarization` as part of the `SummarySkill` is defined and registered with the Semantic Kernel instance.

In [3]:
//Define & register semantic skill
string skPrompt = @"
You are an AI assistant that creates short and accurate summaries of provided text.
The summary must not be more than 2 short sentences. Stop responding after you've created the two sentences. 
Reply in full sentences and avoid bullet points.

---
User: {{$input}}
Assistant: 
---
";

PromptTemplateConfig promptTemplateConfig = new PromptTemplateConfig(){
    Completion = {
        MaxTokens = 2000,
        Temperature = 0.7,
    }
};

PromptTemplate promptTemplate = new PromptTemplate(
    skPrompt,
    promptTemplateConfig,
    kernel
);

string skillName = "SummarySkill";
string functionName = "SimpleSummarization";

SemanticFunctionConfig functionConfig = new SemanticFunctionConfig(promptTemplateConfig, promptTemplate);
kernel.RegisterSemanticFunction(skillName, functionName, functionConfig);



### Step 4: Knowledge Sources

Three Wikipedia articles with knowledge around planets introduced in the Star Wars films are provided. These articles are used as knowledge base which information worker can use as base for providing answers to potential customer questions.

In [4]:
//Define knowledge articles
string[] knowledgeUrls = {
    "https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Dagobah&explaintext=1&exsectionformat=plain", 
    "https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Hoth&explaintext=1&exsectionformat=plain",
    "https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Naboo&explaintext=1&exsectionformat=plain"
};

#!share --from c# knowledgeUrls --as knowledgeUrls

### Step 5: Download knowledge - Summarize knowledge - Store knowledge

Plain c# code is used to download the Wikipedia articles. The registered semantic function `SimpleSummarization`, part of the `SummarySkill`, is used to create a summary of each article.

`kernel.Memory.SaveReferenceAsync()` is used to store the article summary, where the Kernel takes care of the creation of embeddings and the final storage in the registered storage system.

The embeddings are stored in a collection named `StarWarsKnowHow`. 


In [7]:
using System.Net.Http; 

//Download knowledge - Summarize knowledge - Store knowledge (create memory)         
string memoryCollectionName = "StarWarsKnowHow";
foreach (string knowledgeUrl in knowledgeUrls) {

    string knowledge = await DownloadWikiArticle(knowledgeUrl, 2000);
    Console.WriteLine($"Knowledge from {knowledgeUrl} downloaded.");                

    //Create knowledge summary 
    string skill = "SummarySkill";
    string function = "SimpleSummarization";
    ISKFunction skFunction = kernel.Skills.GetFunction(skill, function); 
    ContextVariables contextVariables = new ContextVariables(knowledge);
    string knowledgeSummary = (await kernel.RunAsync(contextVariables, skFunction)).Result;
    Console.WriteLine($"Knowledge Summary created {knowledgeSummary}");

    //Store data (create memory)
    string result = await kernel.Memory.SaveReferenceAsync(
        collection: memoryCollectionName,
        description: knowledgeSummary,
        text: knowledge,
        externalId: knowledgeUrl,
        externalSourceName: "Wikipedia"
    );
    Console.WriteLine($"Knowledge stored");

} 

//Download Wiki article
async Task<string> DownloadWikiArticle(string url, int maxSize = 1000) {

    using HttpClient httpClient = new HttpClient();
    
    HttpResponseMessage httpResponseMessage = await httpClient.GetAsync(url);
    JsonDocument jsonDocument = JsonDocument.Parse(await httpResponseMessage.Content.ReadAsStringAsync());

    JsonElement jsonElement = jsonDocument.RootElement; 
    return IterateJson(jsonElement).Substring(0, maxSize); 

}

string IterateJson(JsonElement root) {

    string article = ""; 

    if (root.ValueKind == JsonValueKind.Array)
    {
        foreach (JsonElement element in root.EnumerateArray())
        {
            IterateJson(element);
        }
    }
    else if (root.ValueKind == JsonValueKind.Object)
    {
        foreach (JsonProperty property in root.EnumerateObject())
        {
            if (property.Name == "extract"){
                article = property.Value.GetString()??"";
                break; 
            }
            article = IterateJson(property.Value);
        }
    }
    return article; 
}

Knowledge from https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Dagobah&explaintext=1&exsectionformat=plain downloaded.
Knowledge Summary created Dagobah is a fictional planet in the Star Wars franchise, known for its murky swamps, steaming bayous, and jungles. It is also noted for being one of the purest places in the galaxy, with a strong living force, and was chosen by Jedi Master Yoda as a place to go into exile.
Knowledge stored
Knowledge from https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Hoth&explaintext=1&exsectionformat=plain downloaded.
Knowledge Summary created Hoth is a small, terrestrial planet covered in snow and ice, home to only a few species including the wampa and tauntauns. It is the location of the Rebel Alliance's secret Echo Base and the site of the "Battle of Hoth" in the 1980 film The Empire Strikes Back.
Knowledge stored
Knowledge from https://en.wikipedia.org/w/api.php?action=query&format=json&p

### Step 6: Query Memories

`kernel.Memory.SearchAsync()` is used to perform semantic queries on stored memories (Wikipedia articles). The query is executed against the collection named `StarWarsKnowHow`. This is the collection where the embeddings have been stored as well.

In [6]:
//Query knowledge
string query = "Where does Joda come from?";
IAsyncEnumerable<MemoryQueryResult> searchResults = kernel.Memory.SearchAsync(memoryCollectionName, query); 
await foreach (MemoryQueryResult memory in searchResults) {
    Console.WriteLine($"{memory.Metadata.Text} : {memory.Relevance} : {memory.Metadata.Id} : {memory.Metadata.Description}");
}

 : 0.7651100978883095 : https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Dagobah&explaintext=1&exsectionformat=plain : Dagobah is a fictional planet in the Star Wars franchise, known for its murky swamps, steaming bayous, and jungles. It is also noted for being one of the purest places in the galaxy, with a strong living force, and was chosen by Jedi Master Yoda as a place to go into exile.
